# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

# import statements
import pandas as pd
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from workspace_utils import active_session
 
from sklearn.linear_model import SGDClassifier

In [4]:
from sklearn.model_selection import GridSearchCV
import pickle

In [5]:
# load data from database
engine = create_engine('sqlite:///emergency.db')
df = pd.read_sql_table('emergency',engine)
df


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [6]:
X = df.message
y=df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Detect URLs
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    # case normalization, punctuation removal and tokenization
    words = word_tokenize(re.sub(r'[^a-zA-Z0-9]', " ", text.lower()))

    # removing stop words
    words = [w for w in words if w not in stopwords.words("english")]

    # lemmatization
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]

    # stemming
    stemmed = [PorterStemmer().stem(w) for w in lemmed]
    return stemmed

In [8]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti'] 

Is the Hurricane over or is it not over
['hurrican'] 

Looking for someone but no name
['look', 'someon', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogan', '80', '90', 'destroy', 'hospit', 'st', 'croix', 'function', 'need', 'suppli', 'desper'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#with active_session():
#     pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
target_names = y.columns


In [9]:
target_names = y.columns
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.71      0.31      0.44      1023
               request       0.72      0.32      0.44      1023
                 offer       0.73      0.31      0.43      1023
           aid_related       0.71      0.31      0.43      1023
          medical_help       0.71      0.31      0.43      1023
      medical_products       0.70      0.28      0.40      1023
     search_and_rescue       0.76      0.31      0.44      1023
              security       0.77      0.30      0.43      1023
              military       0.73      0.33      0.46      1023
           child_alone       0.71      0.31      0.44      1023
                 water       0.73      0.31      0.43      1023
                  food       0.74      0.30      0.43      1023
               shelter       0.77      0.30      0.43      1023
              clothing       0.74      0.30      0.43      1023
                 money       0.74      

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()



parameters = {'clf__estimator__n_estimators': [100, 140]
             
             }

cv = GridSearchCV(pipeline, param_grid=parameters,cv=4, verbose=12, n_jobs=-1)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
with active_session():
     cv.fit(X_train, y_train)

y_pred_cv = cv.predict(X_test)

target_names = y.columns


Fitting 4 folds for each of 2 candidates, totalling 8 fits
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.8187893832346763, total=18.7min
[CV] clf__estimator__n_estimators=100 ................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 21.3min remaining:    0.0s


[CV]  clf__estimator__n_estimators=100, score=0.8143975558525873, total=19.2min
[CV] clf__estimator__n_estimators=100 ................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 43.2min remaining:    0.0s


[CV]  clf__estimator__n_estimators=100, score=0.8147794538858125, total=19.8min
[CV] clf__estimator__n_estimators=100 ................................


...

[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 146.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=140, score=0.8218445675004774, total=26.0min
[CV] clf__estimator__n_estimators=140 ................................


[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed: 175.8min remaining:    0.0s


[CV]  clf__estimator__n_estimators=140, score=0.818025587168226, total=26.2min


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 205.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 205.1min finished


In [14]:
print(classification_report(y_test, y_pred_cv, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.78      0.36      0.50      1023
               request       0.79      0.37      0.50      1023
                 offer       0.79      0.36      0.49      1023
           aid_related       0.79      0.36      0.50      1023
          medical_help       0.78      0.35      0.49      1023
      medical_products       0.79      0.36      0.49      1023
     search_and_rescue       0.79      0.36      0.50      1023
              security       0.78      0.36      0.49      1023
              military       0.77      0.36      0.49      1023
           child_alone       0.78      0.36      0.49      1023
                 water       0.78      0.36      0.49      1023
                  food       0.78      0.37      0.50      1023
               shelter       0.79      0.36      0.49      1023
              clothing       0.79      0.35      0.49      1023
                 money       0.77      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize,max_df=1.0,max_features=None,ngram_range=(1, 2))),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SGDClassifier(loss="hinge",alpha=0.00005,tol=0.001,max_iter=1000)))])


In [19]:
parameters = {'clf__estimator__penalty': ['l1', 'l2']
             
             }

cv = GridSearchCV(pipeline, param_grid=parameters,cv=4, verbose=12, n_jobs=-1)

In [20]:
with active_session():
     cv.fit(X_train,y_train)

Fitting 4 folds for each of 2 candidates, totalling 8 fits
[CV] clf__estimator__penalty=l1 ......................................
[CV]  clf__estimator__penalty=l1, score=0.8185984342180638, total= 1.9min
[CV] clf__estimator__penalty=l1 ......................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.3min remaining:    0.0s


[CV]  clf__estimator__penalty=l1, score=0.8197441283177391, total= 2.0min
[CV] clf__estimator__penalty=l1 ......................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  6.7min remaining:    0.0s


[CV]  clf__estimator__penalty=l1, score=0.8153523009356501, total= 2.0min
[CV] clf__estimator__penalty=l1 ......................................


...

[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 20.4min remaining:    0.0s


[CV]  clf__estimator__penalty=l2, score=0.8512507160588123, total= 2.0min
[CV] clf__estimator__penalty=l2 ......................................


[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed: 23.8min remaining:    0.0s


[CV]  clf__estimator__penalty=l2, score=0.8497231239259118, total= 2.0min


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 27.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 27.2min finished


target_names = y.columns
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [21]:
target_names = y.columns
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.72      0.51      0.59      1023
               request       0.72      0.51      0.60      1023
                 offer       0.71      0.51      0.59      1023
           aid_related       0.72      0.51      0.59      1023
          medical_help       0.72      0.50      0.59      1023
      medical_products       0.72      0.51      0.59      1023
     search_and_rescue       0.72      0.50      0.59      1023
              security       0.72      0.51      0.60      1023
              military       0.71      0.51      0.59      1023
           child_alone       0.72      0.51      0.60      1023
                 water       0.71      0.51      0.59      1023
                  food       0.72      0.51      0.59      1023
               shelter       0.72      0.51      0.60      1023
              clothing       0.72      0.51      0.59      1023
                 money       0.72      

### 9. Export your model as a pickle file

In [17]:


target_names = y.columns
report= classification_report(y_pred_cv,y_test, target_names=target_names)
    
temp=[]
          
for item in report.split("\n"):
    temp.append(item.strip().split('     '))
clean_list=[x for x in temp if x != ['']]
report_df=pd.DataFrame(clean_list[1:],columns=['group','precision','recall', 'f1-score','support'])
report_df

NameError: name 'y_pred_cv' is not defined

In [50]:
with open('model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.